### 1. Simplest Deep Neural Network

In [1]:
import torch
import torch.nn as nn

class DNN1(nn.Module):
    def __init__(self, input_size: int, num_labels: int):
        super().__init__()
        self.classifier = nn.Sequential(
                                        nn.Linear(input_size, 512), nn.ReLU(True),
                                        nn.Linear(512, 512), nn.ReLU(True),
                                        nn.Linear(512, num_labels),
                                        )
    def forward(self, input):
        outputs = self.classifier(input)
        return outputs

In [2]:
import pandas as pd
datapath = "/data/lujd/algorithm2022/audioset/"
dev_df = pd.read_csv(datapath+"dev.csv", sep="\t")

# dev set
import numpy as np
from Extract_feature import extract_mfcc

n_mels = 26
feature_option = "mfcc"
dev_files = list(dev_df.filename)
dev_labels = list(dev_df.scene_label)
dev_features = []
for filename in dev_files:
    wav_file_path = datapath+filename
    mfcc_feature = extract_mfcc(wav_file_path, n_mels=n_mels, option=feature_option)
    dev_features.append(mfcc_feature.reshape(1,-1))             # [1, frames*n_features] (flatten)
dev_features = np.concatenate(dev_features, axis=0)
print(f"dev feature size: {dev_features.shape}")

dev feature size: (379, 6487)


In [3]:
unique_labels = ['airport', 'bus', 'metro', 'metro_station',
                'park', 'public_square', 'shopping_mall',
                'street_pedestrian', 'street_traffic', 'tram']

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = DNN1(dev_features.shape[1], len(unique_labels)).to(device)
model.eval()

batch_size = 128
start_pos, end_pos = 0, 0
with torch.no_grad():
    while end_pos < len(dev_features):
        end_pos = end_pos+batch_size if (end_pos+batch_size)<len(dev_features) else len(dev_features)
        input = torch.Tensor(dev_features[start_pos:end_pos]).to(device)
        print(start_pos, end_pos)
        output = model(input)
        start_pos = end_pos
        print(f"output size: {output.shape}")

0 128
output size: torch.Size([128, 10])
128 256
output size: torch.Size([128, 10])
256 379
output size: torch.Size([123, 10])


In [4]:
output[:5]

tensor([[ 0.0091, -0.0346,  0.1381, -0.4813,  0.0720,  0.0573,  0.3161, -0.2161,
          0.0751, -0.2733],
        [ 0.0067, -0.0158,  0.1083, -0.2737,  0.0704,  0.0213,  0.1434, -0.1082,
          0.0572, -0.1698],
        [ 0.0231, -0.0075,  0.1276, -0.3178,  0.0343,  0.0272,  0.1563, -0.1500,
          0.0561, -0.1808],
        [ 0.0095,  0.0069,  0.0847, -0.2723,  0.0374,  0.0187,  0.1507, -0.1352,
          0.0151, -0.1853],
        [ 0.0011, -0.0277,  0.1109, -0.2673,  0.0473,  0.0580,  0.1613, -0.1307,
          0.0427, -0.1590]], device='cuda:0', grad_fn=<SliceBackward>)

In [5]:
pre_labels = torch.argmax(output, dim=1)
pre_labels[:5]

tensor([6, 6, 6, 6, 6], device='cuda:0')

In [ ]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0005)

In [12]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)           # [3, 5]
target = torch.empty(3, dtype=torch.long).random_(5)    # [3]
output = loss(input, target)
output

tensor(2.7730, grad_fn=<NllLossBackward>)

process data for training DNN

In [24]:
from Extract_feature import extract_mfcc

datapath = "/data/lujd/algorithm2022/audioset/"

class Audio_Dataset(torch.utils.data.Dataset):
    def __init__(self, audio_df, label_dict, feature_type):
        super(Audio_Dataset, self).__init__()
        self.file_list = audio_df.filename.to_list()
        self.label_list = audio_df.scene_label.to_list()
        self.label_dict = label_dict
        
        feature_list = []
        for filename in self.file_list:
            wav_file_path = datapath + filename
            if feature_type=="mfcc" or feature_type=="fbanks":
                feature = extract_mfcc(wav_file_path, option=feature_type)
            feature_list.append(feature.reshape(1,-1))             # [1, frames*n_features] (flatten)
        self.feature_array = np.concatenate(feature_list, axis=0)
        print(self.feature_array.shape)

    def __getitem__(self, index):
        feature, label = self.feature_array[index], self.label_list[index]
        label = self.label_dict[label]
        return feature, label

    def __len__(self):
        return len(self.label_list)

In [25]:
import pandas as pd
dev_df = pd.read_csv(datapath+"dev.csv", sep="\t")

unique_labels = ['airport', 'bus', 'metro', 'metro_station', 'park', 'public_square',
                 'shopping_mall', 'street_pedestrian', 'street_traffic', 'tram']
label_dict = {}
for ind, c in enumerate(unique_labels):
    label_dict[c] = ind

feature_option = "mfcc"
dataset = Audio_Dataset(dev_df, label_dict, feature_option)

(379, 6487)


In [26]:
dev_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=64,
            shuffle=True,
            num_workers=0
        )

In [28]:
for feature_list, label_list in dev_loader:
    print(feature_list[:5])
    break

tensor([[-4.8444,  1.3241,  0.7179,  ...,  0.2825,  0.1520,  0.3084],
        [-8.3584,  2.8215,  0.3826,  ...,  0.2616,  0.0447,  0.1551],
        [-4.2064,  1.8694,  0.6441,  ...,  0.2537, -0.1173, -0.0734],
        [-3.8532,  1.1509, -0.4405,  ...,  0.0849,  0.0238, -0.0204],
        [-2.3669,  2.2008, -0.3398,  ...,  0.1766, -0.0520, -0.1226]],
       dtype=torch.float64)


In [29]:
for feature_list, label_list in dev_loader:
    print(label_list[:5])
    break

tensor([3, 6, 1, 2, 8])


In [66]:
import numpy as np
from Extract_feature import extract_mfcc
import torch.nn as nn
import torch.optim as optim

n_mels = 26
n_frames = 499
n_features = 13

device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
model = DNN1(n_frames*n_features, len(unique_labels)).to(device)
model.train()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.005)

train_loss_list, true_label_list, pre_label_list = [], [], []
for feature_list, label_list in dev_loader:
    
    print(f"dev feature size: {feature_list.shape}")

    # forward
    train_inputs = torch.Tensor(feature_list.float()).to(device)
    train_labels = torch.LongTensor(label_list).to(device)
    train_outputs = model(train_inputs)
    train_loss = criterion(train_outputs, train_labels)

    # backward
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    pre_labels = torch.argmax(train_outputs, dim=1)
    train_loss_list.append(train_loss.cpu().detach().numpy().item())
    true_label_list.append(train_labels.cpu().detach().numpy())
    pre_label_list.append(pre_labels.cpu().detach().numpy())
    
    print(f"output size: {train_outputs.shape}")

dev feature size: torch.Size([64, 6487])
output size: torch.Size([64, 10])
dev feature size: torch.Size([64, 6487])
output size: torch.Size([64, 10])
dev feature size: torch.Size([64, 6487])
output size: torch.Size([64, 10])
dev feature size: torch.Size([64, 6487])
output size: torch.Size([64, 10])
dev feature size: torch.Size([64, 6487])
output size: torch.Size([64, 10])
dev feature size: torch.Size([59, 6487])
output size: torch.Size([59, 10])


In [67]:
train_loss_list

[2.2866268157958984,
 2.2864463329315186,
 2.2715611457824707,
 2.2785751819610596,
 2.2386128902435303,
 2.297173500061035]

In [72]:
true_label_list = np.concatenate(true_label_list)
pre_label_list = np.concatenate(pre_label_list)

In [82]:
true_label_list[:100]

array([4, 6, 8, 8, 1, 9, 5, 0, 6, 6, 3, 7, 9, 3, 3, 5, 4, 0, 5, 6, 2, 1,
       2, 4, 3, 2, 8, 3, 1, 7, 6, 1, 7, 6, 9, 5, 2, 9, 6, 1, 3, 1, 4, 3,
       0, 2, 5, 5, 8, 3, 9, 8, 7, 0, 5, 9, 7, 5, 0, 3, 9, 8, 6, 0, 7, 5,
       3, 0, 6, 8, 7, 2, 9, 2, 7, 8, 2, 9, 8, 3, 6, 5, 8, 5, 8, 1, 0, 4,
       3, 9, 5, 1, 4, 4, 7, 8, 7, 1, 1, 1])

In [81]:
pre_label_list[:100]

array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5])

In [85]:
np.sum(true_label_list == pre_label_list)/len(true_label_list)

0.11609498680738786

In [86]:
len(dev_loader)

6

In [2]:
import numpy as np

a = np.ones(5)
np.expand_dims(a, axis=1)

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [1]:
import numpy as np
a=np.array([[[1,2],[3,4]],[[5,6],[7,8]]])
a

array([[[1, 2],
        [3, 4]],

       [[5, 6],
        [7, 8]]])

In [2]:
a.reshape(2,-1)

array([[1, 2, 3, 4],
       [5, 6, 7, 8]])

In [1]:
import torch
a=torch.LongTensor([1,2,3,4,5])
a

tensor([1, 2, 3, 4, 5])

In [3]:
b=a.repeat((10,1))
b

tensor([[1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5]])

In [7]:
b.permute(1,0).reshape(-1)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5])

In [11]:
c=a.repeat((10,1)).permute(1,0).reshape(-1).numpy()
c.reshape(-1,10)

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
       [3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
       [4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
       [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]])